In [90]:
import os
import pandas as pd
import numpy as np
import copy

In [93]:
cn_dict = np.load("data/cn_dict.npy",allow_pickle='TRUE').item()
ad_dict = np.load("data/ad_dict.npy",allow_pickle='TRUE').item()
smci_dict = np.load("data/smci_dict.npy",allow_pickle='TRUE').item()
pmci_dict = np.load("data/pmci_dict.npy",allow_pickle='TRUE').item()

In [7]:
IMAGES = pd.read_csv("data/Big_cohort_MRI_Images_25Oct2024.csv")
IMAGES

,image_id,subject_id,mri_visit,mri_date,mri_description,mri_type,mri_weighting,mri_sequence,mri_thickness,mri_te,...,mri_flip_angle,mri_acq_plane,mri_width,mri_height,mri_n_images,mri_pixel_x,mri_pixel_y,mri_mfr,mri_mfr_model,mri_field_str
0,70846,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,AXIAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
1,70847,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,CORONAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
2,70848,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,SAGITTAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
3,70849,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,AXIAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
4,70850,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,CORONAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115989,10967970,011_S_10261,4_sc,2024-10-02,Axial 3D ME T2 GRE (MSV21),3D,T2,GR,1.8,18.440001,...,15,AXIAL,384.0,384.0,88,0.52083,0.52083,SIEMENS,Prisma_fit,3.0
115990,10967971,011_S_10261,4_sc,2024-10-02,Axial 3D ME T2 GRE (MSV21),3D,T2,GR,1.8,10.620000,...,15,AXIAL,384.0,384.0,88,0.52083,0.52083,SIEMENS,Prisma_fit,3.0
115991,10967972,011_S_10261,4_sc,2024-10-02,Axial 3D ME T2 GRE (MSV21),3D,T2,GR,1.8,6.710000,...,15,AXIAL,384.0,384.0,88,0.52083,0.52083,SIEMENS,Prisma_fit,3.0
115992,10967973,011_S_10261,4_sc,2024-10-02,Axial 3D ME T2 GRE (MSV21),3D,T2,GR,1.8,22.350000,...,15,AXIAL,384.0,384.0,88,0.52083,0.52083,SIEMENS,Prisma_fit,3.0


In [109]:
image_dict = {}

for index, row in IMAGES.iterrows():

    #Add filters to decide what images to include.
    if row['mri_type'] == '3D':

        if row['subject_id'] in image_dict:
            image_dict[row['subject_id']].append([row['mri_visit'], row['image_id']])
        else:
            image_dict[row['subject_id']] = [[row['mri_visit'], row['image_id']]]

In [110]:
image_dict

{'037_S_1421': [['sc', 70854],
  ['sc', 70855],
  ['m06', 91145],
  ['m06', 91146],
  ['m12', 118124],
  ['m12', 118125],
  ['m18', 143718],
  ['m18', 143719],
  ['m36', 193596],
  ['m36', 193604]],
 '127_S_1427': [['sc', 69349],
  ['sc', 69350],
  ['sc', 69355],
  ['sc', 69356],
  ['m06', 91501],
  ['m06', 91502],
  ['m06', 91510],
  ['m06', 91511],
  ['m12', 117504],
  ['m12', 117505],
  ['m12', 117510],
  ['m12', 117511],
  ['m24', 153729],
  ['m24', 153735],
  ['m18', 138999],
  ['m18', 139000],
  ['m18', 139005],
  ['m18', 139006],
  ['m24', 153734],
  ['m24', 153728],
  ['m36', 191028],
  ['m36', 191029],
  ['m36', 191032],
  ['m36', 191035],
  ['v06', 251885],
  ['v06', 251888],
  ['v11', 323862],
  ['v11', 323867],
  ['v21', 385969],
  ['v21', 385974],
  ['v31', 439895],
  ['v31', 439899],
  ['v51', 765629],
  ['v51', 765632],
  ['init', 901020],
  ['init', 901026],
  ['init', 901027],
  ['init', 901029],
  ['init', 901030],
  ['y1', 1059953],
  ['y1', 1059957],
  ['y1', 105995

In [111]:
def combine_patients_pictures(in_dict):  

    dict_with_images = copy.deepcopy(in_dict)

    for patient in dict_with_images:

        if patient in image_dict:
            visits_pictures = image_dict[patient]

            for visit in dict_with_images[patient]:
                visit.append(list())

                for picture in visits_pictures: 

                    if visit[0] == picture[0]:
                        visit[3].append(picture[1])
        else:
            pass

    return dict_with_images


In [112]:
cn_dict_with_images = combine_patients_pictures(cn_dict)
ad_dict_with_images = combine_patients_pictures(ad_dict)
smci_dict_with_images = combine_patients_pictures(smci_dict)
pmci_dict_with_images = combine_patients_pictures(pmci_dict)


print(cn_dict_with_images)
print(ad_dict_with_images)
print(smci_dict_with_images)
print(pmci_dict_with_images)

{'011_S_0005': [['bl', '2005-09-30', 1.0, []], ['m06', '2006-03-09', 1.0, [11735, 11736, 11738, 11739]], ['m12', '2006-09-06', 1.0, [24618, 24617, 24615, 24614]], ['m24', '2007-09-07', 1.0, [193493, 193495, 193496, 193500]], ['m36', '2008-09-10', 1.0, [117773, 117774, 117776, 117777]]], '023_S_0031': [['bl', '2005-10-24', 1.0, [8299, 8298, 8296, 8295]], ['m06', '2006-04-18', 1.0, [13531, 13532, 13535, 13536, 13557, 13558, 13560, 13561]], ['m12', '2006-10-30', 1.0, [28012, 28013, 28018, 28019, 28026, 28027, 28029, 28030]], ['m24', '2007-11-20', 1.0, [82185, 82186, 82190, 82191, 82250, 82251, 82254, 82255]], ['m36', '2008-12-02', 1.0, [128975, 128976, 128978, 128979, 128980, 129036, 129037, 129040, 129041]], ['m48', '2009-10-23', 1.0, [157579, 157580, 157583, 157584]], ['m60', '2010-12-10', 1.0, [207217, 207219, 207220, 207222]], ['m72', '2011-10-23', 1.0, []], ['m84', '2012-09-25', 1.0, []], ['m96', '2013-10-21', 1.0, []], ['m120', '2016-04-11', 1.0, []], ['m150', '2018-04-25', 1.0, []]